In [106]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.preprocessing import LabelEncoder
import pickle

In [107]:
# loading the data from csv file to pandas dataframe
df = pd.read_csv('/Users/gokul/Documents/Python/Data/autos.csv',header=0,sep=',', encoding="Latin1")

In [108]:
df.shape

(371528, 18)

In [109]:
#checking the number of rows and columns
df.head()

,dateCrawled,name,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÃRER,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [110]:
#Cars having power less than 50ps and above 900ps seems a little suspicious,
#let's remove them and see what we've got now
print (df.shape)

(371528, 18)


In [111]:
df = df[(df.powerPS > 50) & (df.powerPS < 900)]
print (df.shape)

(319709, 18)


In [112]:
df = df[ (df.yearOfRegistration >= 1950) & (df.yearOfRegistration < 2017) ]

In [113]:
#simlarly,
#simlarly, filtering our the cars having registration years not in the mentioned range
print (df.shape)
# not much of a difference but still, 10000 rows have been reduced. it's better to
#get rid of faulty data instead of keeping them just to increase the size.

(309171, 18)


In [114]:
#removing irrelevant columns which are either the same for all the cars in teh dataset, or can
#introduce bias, so removing them too.

df. drop ([ 'name','abtest','dateCrawled','nrOfPictures',
          'postalCode','dateCreated','lastSeen'], axis='columns', inplace=True)

In [115]:
df

,price,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
1,18300,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja
2,9800,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN
3,1500,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein
4,3600,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein
5,650,limousine,1995,manuell,102,3er,150000,10,benzin,bmw,ja
...,...,...,...,...,...,...,...,...,...,...,...
371520,3200,limousine,2004,manuell,225,leon,150000,5,benzin,seat,ja
371524,1199,cabrio,2000,automatik,101,fortwo,125000,3,benzin,smart,nein
371525,9200,bus,1996,manuell,102,transporter,150000,3,diesel,volkswagen,nein
371526,3400,kombi,2002,manuell,100,golf,150000,6,diesel,volkswagen,NaN


In [116]:
#dropping the duplicates from the dataframe and string it in a new df
#here all rows having same value in all the mentioned columns will be deleted and by default.
#onlv first occurance of anvsuch row is kept
new_df = df.copy()
new_df = new_df.drop_duplicates(['price', 'vehicleType','yearOfRegistration',
                                 'gearbox','powerPS','model','kilometer',
                                 'monthOfRegistration', 'fuelType','notRepairedDamage'])

In [76]:
new_df.head()

,price,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
1,18300,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja
2,9800,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN
3,1500,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein
4,3600,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein
5,650,limousine,1995,manuell,102,3er,150000,10,benzin,bmw,ja


In [117]:
#As the dataset contained some german words for many features, cahnging them to english
new_df.gearbox.replace(( 'manuell','automatik'),('manual','automatic'),inplace=True)
new_df.fuelType.replace(('benzin','andere','elektro'),('petrol','others','electric'), inplace=True)
new_df.vehicleType.replace(('kleinwagen','cabrio','kombi','andere'),('small car','convertible','combination','others'),inplace=True)
new_df.notRepairedDamage.replace(('ja','nein'),('Yes','No'), inplace=True)

In [118]:
new_df.head()

,price,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
1,18300,coupe,2011,manual,190,NaN,125000,5,diesel,audi,Yes
2,9800,suv,2004,automatic,163,grand,125000,8,diesel,jeep,NaN
3,1500,small car,2001,manual,75,golf,150000,6,petrol,volkswagen,No
4,3600,small car,2008,manual,69,fabia,90000,7,diesel,skoda,No
5,650,limousine,1995,manual,102,3er,150000,10,petrol,bmw,Yes


In [119]:
#### Removing the outliers
new_df = new_df[ (new_df.price >= 100) & (new_df.price <= 150000) ]

In [120]:
#Filling NaN values for columns whose data might not be there with the information provider,
#which might lead to some variance but our model
#but we will still be able to give some estimate to the user
new_df['notRepairedDamage'].fillna(value='not-declared',inplace=True)
new_df['fuelType'].fillna(value='not-declared',inplace=True)
new_df[ 'gearbox'].fillna(value='not-declared', inplace=True)
new_df[ 'vehicleType'].fillna (value='not-declared', inplace=True)
new_df['model'].fillna(value='not-declared', inplace=True)

In [121]:
new_df.head()

,price,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
1,18300,coupe,2011,manual,190,not-declared,125000,5,diesel,audi,Yes
2,9800,suv,2004,automatic,163,grand,125000,8,diesel,jeep,not-declared
3,1500,small car,2001,manual,75,golf,150000,6,petrol,volkswagen,No
4,3600,small car,2008,manual,69,fabia,90000,7,diesel,skoda,No
5,650,limousine,1995,manual,102,3er,150000,10,petrol,bmw,Yes


In [122]:
#can save the cv for future purpose.
new_df.to_csv("autos_preprocessed.csv")

In [123]:
#Columns which contain categorical values, which we'll need to convert via label encoding
labels = ['gearbox','notRepairedDamage','model','brand','fuelType'
,'vehicleType']

In [124]:
#looping over the labels to do the label encoding for all at once and
#saving the LABEL ENCODING FILES
mapper = {}
for i in labels:
    mapper[i] = LabelEncoder()
    mapper[i].fit(new_df[i])
    tr=mapper[i].transform(new_df[i])
    np.save(str('classes'+i+'.npy'), mapper[i].classes_)
    print(i,":", mapper[i])
    new_df.loc[:, i+'_labels'] = pd.Series(tr,index=new_df.index)

gearbox : LabelEncoder()
notRepairedDamage : LabelEncoder()
model : LabelEncoder()
brand : LabelEncoder()
fuelType : LabelEncoder()
vehicleType : LabelEncoder()


In [125]:
#Final data to be put in a new dataframe called "LABELED"
labeled = new_df[ ['price','yearOfRegistration','powerPS','kilometer'
,'monthOfRegistration']
                  + [x + "_labels" for x in labels]]

In [126]:
print (labeled.columns)

Index(['price', 'yearOfRegistration', 'powerPS', 'kilometer',
       'monthOfRegistration', 'gearbox_labels', 'notRepairedDamage_labels',
       'model_labels', 'brand_labels', 'fuelType_labels',
       'vehicleType_labels'],
      dtype='object')


In [127]:
new_df.shape

(278578, 17)

In [128]:
new_df

,price,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,gearbox_labels,notRepairedDamage_labels,model_labels,brand_labels,fuelType_labels,vehicleType_labels
1,18300,coupe,2011,manual,190,not-declared,125000,5,diesel,audi,Yes,1,1,162,1,1,3
2,9800,suv,2004,automatic,163,grand,125000,8,diesel,jeep,not-declared,0,2,118,14,1,8
3,1500,small car,2001,manual,75,golf,150000,6,petrol,volkswagen,No,1,0,117,38,7,7
4,3600,small car,2008,manual,69,fabia,90000,7,diesel,skoda,No,1,0,102,31,1,7
5,650,limousine,1995,manual,102,3er,150000,10,petrol,bmw,Yes,1,1,11,2,7,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371520,3200,limousine,2004,manual,225,leon,150000,5,petrol,seat,Yes,1,1,141,30,7,4
371524,1199,convertible,2000,automatic,101,fortwo,125000,3,petrol,smart,No,0,0,107,32,7,2
371525,9200,bus,1996,manual,102,transporter,150000,3,diesel,volkswagen,No,1,0,224,38,1,0
371526,3400,combination,2002,manual,100,golf,150000,6,diesel,volkswagen,not-declared,1,2,117,38,1,1


In [129]:
#Storing price in Y and rest of the data in X
Y = labeled.iloc[:,0].values
X = labeled.iloc[:,1:].values

In [130]:
#need to reshape the Y values
Y= Y.reshape(-1,1)

In [131]:
from sklearn.model_selection import cross_val_score, train_test_split

In [132]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state = 3)

In [133]:
#Model building and Fitting
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
regressor = RandomForestRegressor (n_estimators=1000, max_depth=10, random_state=34)

In [134]:
#fitting the model
regressor.fit(X_train, np.ravel(Y_train, order='C'))

RandomForestRegressor(max_depth=10, n_estimators=1000, random_state=34)

In [135]:
#predicting the values fo test test
y_pred = regressor.predict(X_test)

In [136]:
#printing the Accuracy for test set
print(r2_score(Y_test,y_pred))

0.834527626497731


In [137]:
#saving the model for future use.
filename = 'resale_model.sav'
pickle.dump(regressor, open(filename,'wb'))

In [ ]:
if __name__=="__main__":
    app.run(debug=True)